# CrewAI Coder Agent - Control Center

This notebook provides an interactive interface to control and monitor the CrewAI coder agent.

## Features
- Configure and run the agent
- Monitor execution in real-time
- View logs and metrics
- Customize prompts and settings
- Inspect generated files

## Setup and Imports

In [1]:
# Imports
import sys
import time
from datetime import datetime
from pathlib import Path

from helpers.logger import SessionLogger
from helpers.config_loader import load_config
from core.agent_factory import create_coder_crew
from core.context_loader import load_prompt_template

print("✓ All imports successful")

✓ All imports successful


## Load Configuration

In [2]:
# Load config
config = load_config()

print("Configuration:")
print(f"  LLM Model: {config.get('llm.model')}")
print(f"  Base URL: {config.get('llm.base_url')}")
print(f"  Temperature: {config.get('llm.temperature')}")
print(f"  Max Tokens: {config.get('llm.max_tokens')}")
print(f"  Agent Role: {config.get('agent.role')}")
print(f"  Max Iterations: {config.get('agent.max_iterations')}")

Configuration:
  LLM Model: qwen/qwen3-coder-30b
  Base URL: http://localhost:1234/v1
  Temperature: 0.7
  Max Tokens: 4000
  Agent Role: Senior Full-Stack Developer
  Max Iterations: 25


## Initialize Session Logger

In [3]:
# Create session logger
session_id = f"notebook_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
logger = SessionLogger(session_id=session_id)

logger.print_success(f"Session started: {session_id}")
logger.print_info(f"Logs will be saved to: logs/{session_id}.log")

✓ Session started: notebook_20251109_154833

ℹ Logs will be saved to: logs/notebook_20251109_154833.log

## View Default Prompt

In [4]:
# Load and display the default prompt
default_prompt = load_prompt_template('build-website')
print("Default Task Prompt:")
print("=" * 80)
print(default_prompt[:1000])  # Show first 1000 characters
print("...")
print(f"\nTotal length: {len(default_prompt)} characters")

Default Task Prompt:
# Build Personal Website - Agent Task

## Objective
Create a modern, responsive personal portfolio website using React and Tailwind CSS.

## Requirements

### Technical Stack
- React 18+
- Tailwind CSS for styling
- Vite as build tool
- Modern ES6+ JavaScript
- Responsive design (mobile-first)

### Features to Implement

#### 1. Navigation
- Sticky navigation bar
- Smooth scroll to sections
- Mobile hamburger menu
- Active section highlighting

#### 2. Hero Section
- Full-screen hero with gradient background
- Name and title/tagline
- Call-to-action button
- Subtle animations on load

#### 3. About Section
- Personal introduction (2-3 paragraphs)
- Professional background
- Current work/interests
- Optional: profile image with hover effect

#### 4. Skills Section
- Grid layout of technical skills
- Categorized skills (Frontend, Backend, Tools, etc.)
- Visual indicators (icons or badges)
- Hover effects

#### 5. Portfolio/Projects Section
- Grid of project cards (mi

## Option 1: Run with Default Prompt

In [5]:
# Create and run crew with default settings
logger.print_section("Creating CrewAI Crew")

crew = create_coder_crew(verbose=True)

logger.print_success("Crew created successfully")
logger.print_info(f"Agents: {len(crew.agents)}")
logger.print_info(f"Tasks: {len(crew.tasks)}")

────────────────────────────────────────────── Creating CrewAI Crew ───────────────────────────────────────────────

✓ Crew created successfully

ℹ Agents: 1

ℹ Tasks: 1

In [6]:
# Execute the crew
logger.print_section("Executing Task")
logger.print_info("Building personal website...")
logger.print_info("This may take several minutes.\n")

start_time = time.time()

try:
    result = crew.kickoff()
    
    execution_time = time.time() - start_time
    
    logger.print_section("Execution Complete")
    logger.print_success(f"Completed in {execution_time:.2f} seconds")
    
    print("\nResult:")
    print("=" * 80)
    print(result)
    
except Exception as e:
    logger.print_error(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()

───────────────────────────────────────────────── Executing Task ──────────────────────────────────────────────────

ℹ Building personal website...

ℹ This may take several minutes.

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Full-Stack Developer                                                                             │
│                                                                                                                 │
│  Task: # Build Personal Website - Agent Task                                                                    │
│                                                                                                                 │
│  ## Objective                                                                                                   │
│  Create a modern, responsive personal portfolio website using React and Tailwind CSS.                           │
│                                                                                                                 │
│  ## Requirements                                                                                                │
│                                                                                                                 │
│  ### Technical Stack                                                                                            │
│  - React 18+                                                                                                    │
│  - Tailwind CSS for styling                                                                                     │
│  - Vite as build tool                                                                                           │
│  - Modern ES6+ JavaScript                                                                                       │
│  - Responsive design (mobile-first)                                                                             │
│                                                                                                                 │
│  ### Features to Implement                                                                                      │
│                                                                                                                 │
│  #### 1. Navigation                                                                                             │
│  - Sticky navigation bar                                                                                        │
│  - Smooth scroll to sections                                                                                    │
│  - Mobile hamburger menu                                                                                        │
│  - Active section highlighting                                                                                  │
│                                                                                                                 │
│  #### 2. Hero Section                                                                                           │
│  - Full-screen hero with gradient background                                                                    │
│  - Name and title/tagline                                                                                       │
│  - Call-to-action button                                                                                        │
│  - Subtle animations on load                                                                                    │
│                                                                                                                 │
│  #### 3. About Section                                                                                          │
│  - Personal introduction (2-3 paragraphs)                                                                       │
│  - Professional background                             

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 822e46c9-161d-42fd-b8ba-de5dae662616                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Received None or empty response from LLM call.
An unknown error occurred. Please check the details below.
Error details: Invalid response from LLM call - None or empty.
An unknown error occurred. Please check the details below.
Error details: Invalid response from LLM call - None or empty.


Output()

✗ Error: Invalid response from LLM call - None or empty.

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 9c62e048-fcec-442e-97cd-ffae0eef38e8                                                                     │
│  Agent: Senior Full-Stack Developer                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Traceback (most recent call last):
  File "/Users/Khaled.Alabsi/projects/llm-agent/.venv/lib/python3.13/site-packages/crewai/agent/core.py", line 489, in execute_task
    result = self._execute_without_timeout(task_prompt, task)
  File "/Users/Khaled.Alabsi/projects/llm-agent/.venv/lib/python3.13/site-packages/crewai/agent/core.py", line 591, in _execute_without_timeout
    return self.agent_executor.invoke(
           ~~~~~~~~~~~~~~~~~~~~~~~~~~^
        {
        ^
    ...<4 lines>...
        }
        ^
    )["output"]
    ^
  File "/Users/Khaled.Alabsi/projects/llm-agent/.venv/lib/python3.13/site-packages/crewai/agents/crew_agent_executor.py", line 180, in invoke
    formatted_answer = self._invoke_loop()
  File "/Users/Khaled.Alabsi/projects/llm-agent/.venv/lib/python3.13/site-packages/crewai/agents/crew_agent_executor.py", line 289, in _invoke_loop
    raise e
  File "/Users/Khaled.Alabsi/projects/llm-agent/.venv/lib/python3.13/site-packages/crewai/agents/crew_agent_executor.py", 

## Option 2: Run with Custom Prompt

In [7]:
# Define custom prompt
custom_prompt = """
Create a minimalist personal portfolio website with the following requirements:

Technical Stack:
- React 18 with functional components
- Tailwind CSS for styling
- Vite as build tool

Sections:
1. Hero section with name and title
2. About section
3. Skills grid
4. Project showcase (3 projects minimum)
5. Contact form

Requirements:
- Fully responsive design
- Dark mode toggle
- Smooth scrolling navigation
- Modern, clean aesthetic
- All code must be well-documented
- Include package.json and README

Output everything to ./output directory.
"""

print("Custom Prompt:")
print(custom_prompt)

Custom Prompt:

Create a minimalist personal portfolio website with the following requirements:

Technical Stack:
- React 18 with functional components
- Tailwind CSS for styling
- Vite as build tool

Sections:
1. Hero section with name and title
2. About section
3. Skills grid
4. Project showcase (3 projects minimum)
5. Contact form

Requirements:
- Fully responsive design
- Dark mode toggle
- Smooth scrolling navigation
- Modern, clean aesthetic
- All code must be well-documented
- Include package.json and README

Output everything to ./output directory.



In [8]:
# Create crew with custom prompt
custom_crew = create_coder_crew(custom_prompt=custom_prompt, verbose=True)

logger.print_success("Custom crew created")

✓ Custom crew created

In [ ]:
# Execute custom crew
start_time = time.time()

try:
    result = custom_crew.kickoff()
    
    execution_time = time.time() - start_time
    logger.print_success(f"Custom task completed in {execution_time:.2f} seconds")
    
    print(result)
    
except Exception as e:
    logger.print_error(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 844df0ff-8b59-40d8-b8f4-9c82e52d398a                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Full-Stack Developer                                                                             │
│                                                                                                                 │
│  Task:                                                                                                          │
│  Create a minimalist personal portfolio website with the following requirements:                                │
│                                                                                                                 │
│  Technical Stack:                                                                                               │
│  - React 18 with functional components                                                                          │
│  - Tailwind CSS for styling                                                                                     │
│  - Vite as build tool                                                                                           │
│                                                                                                                 │
│  Sections:                                                                                                      │
│  1. Hero section with name and title                                                                            │
│  2. About section                                                                                               │
│  3. Skills grid                                                                                                 │
│  4. Project showcase (3 projects minimum)                                                                       │
│  5. Contact form                                                                                                │
│                                                                                                                 │
│  Requirements:                                                                                                  │
│  - Fully responsive design                                                                                      │
│  - Dark mode toggle                                                                                             │
│  - Smooth scrolling navigation                                                                                  │
│  - Modern, clean aesthetic                                                                                      │
│  - All code must be well-documented                                                                             │
│  - Include package.json and README                                                                              │
│                                                                                                                 │
│  Output everything to ./output directory.                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Full-Stack Developer                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```                                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✓ Custom task completed in 2.32 seconds

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0cff93d7-291b-4bf1-8bc2-8de1f5d12f76                                                                     │
│  Agent: Senior Full-Stack Developer                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Inspect Generated Files

In [10]:
# List all files in output directory
import os

output_dir = Path("./output")

if output_dir.exists():
    print("Generated Files:")
    print("=" * 80)
    
    for root, dirs, files in os.walk(output_dir):
        level = root.replace(str(output_dir), '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            file_path = Path(root) / file
            size = file_path.stat().st_size
            print(f"{subindent}{file} ({size:,} bytes)")
else:
    print("Output directory not found or empty")

Generated Files:
output/
  .gitkeep (131 bytes)
  output/
    personal-website/
      public/
      src/
        styles/
        components/
        data/


## View Specific Files

In [11]:
# Read and display a specific file
file_to_view = "output/package.json"  # Change this to view different files

try:
    with open(file_to_view, 'r') as f:
        content = f.read()
    
    print(f"File: {file_to_view}")
    print("=" * 80)
    print(content)
    
except FileNotFoundError:
    print(f"File not found: {file_to_view}")
except Exception as e:
    print(f"Error reading file: {e}")

File not found: output/package.json


## View Session Logs

In [12]:
# View session metrics
import json

json_log = Path(f"logs/{session_id}.json")

if json_log.exists():
    with open(json_log, 'r') as f:
        session_data = json.load(f)
    
    print("Session Metrics:")
    print("=" * 80)
    print(json.dumps(session_data.get('metrics', {}), indent=2))
    
    print("\nRecent Events:")
    print("=" * 80)
    events = session_data.get('events', [])
    for event in events[-5:]:  # Show last 5 events
        print(f"\n[{event['timestamp']}] {event['type']}")
        if 'data' in event:
            print(json.dumps(event['data'], indent=2)[:200])
else:
    print("Session log not found yet")

Session Metrics:
{
  "total_llm_calls": 0,
  "total_tool_calls": 0,
  "total_tokens": 0,
  "errors": 0
}

Recent Events:

[2025-11-09T15:48:33.590512] session_start
{
  "session_id": "notebook_20251109_154833"
}


## End Session

In [13]:
# End session and save final logs
logger.end_session()

print("\n✓ Session ended. Logs saved.")

───────────────────────────────────────────────── Session Summary ─────────────────────────────────────────────────

Session ID: notebook_20251109_154833

Duration: 335.34 seconds

LLM Calls: 0

Tool Calls: 0

Total Tokens: 0

Errors: 0

Logs saved to: logs/notebook_20251109_154833.log


✓ Session ended. Logs saved.


## Quick Reference

### Common Tasks

1. **Run with default settings**: Execute the first "Option 1" cells
2. **Run with custom prompt**: Modify and execute "Option 2" cells
3. **Check output**: Run "Inspect Generated Files" cell
4. **View logs**: Run "View Session Logs" cell

### Configuration Changes

Edit `config.yaml` to change:
- LLM settings (model, temperature, etc.)
- Agent behavior (role, goal, max iterations)
- Output directory
- Logging settings

### Context Files

Modify these to change agent behavior:
- `context/no-goes.md` - Safety rules
- `context/guidelines/coding-standards.md` - Code standards
- `context/skills/` - Knowledge base
- `prompts/build-website.md` - Default task prompt

### Output Directory

All generated files go to `./output/`
This directory is gitignored but tracked for changes.

### Logs Directory

Session logs are saved to `./logs/`
- `session_*.log` - Human-readable log
- `session_*.json` - Structured data with metrics


In [14]:
# Cell 1: Imports
import sys
import time
from datetime import datetime
from pathlib import Path

from helpers.logger import SessionLogger
from helpers.config_loader import load_config
from core.agent_factory import create_coder_crew
from core.context_loader import load_prompt_template

print("✓ All imports successful")

✓ All imports successful


In [15]:
# Cell 2: Load Configuration
config = load_config()

print("Configuration:")
print(f"  LLM Model: {config.get('llm.model')}")
print(f"  Base URL: {config.get('llm.base_url')}")
print(f"  Temperature: {config.get('llm.temperature')}")
print(f"  Max Tokens: {config.get('llm.max_tokens')}")
print(f"  Agent Role: {config.get('agent.role')}")
print(f"  Max Iterations: {config.get('agent.max_iterations')}")

Configuration:
  LLM Model: qwen/qwen3-coder-30b
  Base URL: http://localhost:1234/v1
  Temperature: 0.7
  Max Tokens: 4000
  Agent Role: Senior Full-Stack Developer
  Max Iterations: 25


In [16]:
# Cell 2: Load Configuration
config = load_config()

print("Configuration:")
print(f"  LLM Model: {config.get('llm.model')}")
print(f"  Base URL: {config.get('llm.base_url')}")
print(f"  Temperature: {config.get('llm.temperature')}")
print(f"  Max Tokens: {config.get('llm.max_tokens')}")
print(f"  Agent Role: {config.get('agent.role')}")
print(f"  Max Iterations: {config.get('agent.max_iterations')}")

Configuration:
  LLM Model: qwen/qwen3-coder-30b
  Base URL: http://localhost:1234/v1
  Temperature: 0.7
  Max Tokens: 4000
  Agent Role: Senior Full-Stack Developer
  Max Iterations: 25


In [17]:
# Cell 3: Initialize Session Logger
session_id = f"notebook_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
logger = SessionLogger(session_id=session_id)

logger.print_success(f"Session started: {session_id}")
logger.print_info(f"Logs will be saved to: logs/{session_id}.log")

✓ Session started: notebook_20251109_155408

ℹ Logs will be saved to: logs/notebook_20251109_155408.log

In [18]:
# Cell 4: View Default Prompt
default_prompt = load_prompt_template('build-website')
print("Default Task Prompt:")
print("=" * 80)
print(default_prompt[:1000])  # Show first 1000 characters
print("...")
print(f"\nTotal length: {len(default_prompt)} characters")

Default Task Prompt:
# Build Personal Website - Agent Task

## Objective
Create a modern, responsive personal portfolio website using React and Tailwind CSS.

## Requirements

### Technical Stack
- React 18+
- Tailwind CSS for styling
- Vite as build tool
- Modern ES6+ JavaScript
- Responsive design (mobile-first)

### Features to Implement

#### 1. Navigation
- Sticky navigation bar
- Smooth scroll to sections
- Mobile hamburger menu
- Active section highlighting

#### 2. Hero Section
- Full-screen hero with gradient background
- Name and title/tagline
- Call-to-action button
- Subtle animations on load

#### 3. About Section
- Personal introduction (2-3 paragraphs)
- Professional background
- Current work/interests
- Optional: profile image with hover effect

#### 4. Skills Section
- Grid layout of technical skills
- Categorized skills (Frontend, Backend, Tools, etc.)
- Visual indicators (icons or badges)
- Hover effects

#### 5. Portfolio/Projects Section
- Grid of project cards (mi

In [19]:
# Cell 5: Create CrewAI Crew
logger.print_section("Creating CrewAI Crew")

crew = create_coder_crew(verbose=True)

logger.print_success("Crew created successfully")
logger.print_info(f"Agents: {len(crew.agents)}")
logger.print_info(f"Tasks: {len(crew.tasks)}")

────────────────────────────────────────────── Creating CrewAI Crew ───────────────────────────────────────────────

✓ Crew created successfully

ℹ Agents: 1

ℹ Tasks: 1